In [10]:
import zipfile
import os
import pandas as pd
import glob

# Step 1: Extract the Zip File
zip_file_path = 'Property_data.zip'  # Replace with your zip file's path
extraction_path = 'Property_data/Property_data'  # Directory where files will be extracted

# Extract the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extraction_path)

print("Zip file extracted successfully.")

# Step 2: Check if CSV Files Are Present
csv_files_path = os.path.join(extraction_path, '*.csv')

# Get a list of all CSV file paths
csv_files = glob.glob(csv_files_path)

# Debug: Check how many CSV files were found
print(f"Found {len(csv_files)} CSV files at {csv_files_path}.")
if len(csv_files) == 0:
    print("No CSV files found. Please check the extraction path and zip file contents.")
else:
    # Step 3: List Extracted Files (Debugging)
    extracted_files = os.listdir(extraction_path)
    print(f"Files extracted: {extracted_files}")

    # Step 4: Merge All CSV Files
    dataframes = []  # Initialize an empty list to store DataFrames

    # Loop through each CSV file and append to the list
    for file in csv_files:
        try:
            # Try reading the file with a different encoding to avoid issues like 'ÿþ'
            df = pd.read_csv(file, encoding='unicode_escape')  # Fallback encoding
            print(f"File: {file}, Shape: {df.shape}")
            print(f"Columns in {file}: {df.columns.tolist()}")

            # Check if the necessary 'property_id' column exists before appending
            if 'property_id' in df.columns:
                dataframes.append(df)
            else:
                print(f"'property_id' column NOT found in {file}")

        except Exception as e:
            print(f"Error reading {file}: {e}")

    # Step 5: Concatenate all DataFrames into one
    if len(dataframes) > 0:
        merged_data = pd.concat(dataframes, ignore_index=True)

        # Step 6: Clean and Standardize the Data
        merged_data.drop_duplicates(inplace=True)  # Drop duplicates

        # Handle missing values (fill with default values)
        merged_data.fillna({
            'property_id': 'Unknown',
            'type': 'Unknown',
            'activation_date': 'Unknown',
            'bathroom': 0,
            'floor': 0,
            'total_floor': 0,
            'furnishing': 'Unknown',
            'gym': 0,
            'latitude': 0,
            'longitude': 0,
            'lease_type': 'Unknown',
            'lift': 0,
            'locality': 'Unknown',
            'parking': 'Unknown',
            'property_age': 0,
            'property_size': 0,
            'swimming_pool': 0,
            'pin_code': 'Unknown',
            'rent': 0,
            'deposit': 0,
            'building_type': 'Unknown',
            'location': 'Unknown'
        }, inplace=True)

        # Remove unwanted columns like the one starting with 'ÿþ'
        merged_data = merged_data.loc[:, ~merged_data.columns.str.contains('ÿþ')]

        # Step 7: Save the Merged Data
        output_file = 'merged_property_data.csv'
        merged_data.to_csv(output_file, index=False)
        print(f"Merged data saved to {output_file}")

        # Step 8: Print Final Data Information
        print(f"Final merged data contains {merged_data.shape[0]} rows and {merged_data.shape[1]} columns.")
        print("Missing values in each column:")
        print(merged_data.isnull().sum())
    else:
        print("No data loaded from CSV files.")

# Step 9: Define file path and load cleaned data
input_dir = "Property_data/Property_data"  # Replace with your actual directory path
output_dir = "Property_data/outputs"  # Replace with your desired output directory

# Step 10: Load the cleaned merged data
merged_data = pd.read_csv("Property_data/outputs/cleaned_merged_property_data_with_imputation.csv")

# Remove the unwanted 'Unnamed' column if it exists
merged_data = merged_data.loc[:, ~merged_data.columns.str.contains('^Unnamed')]

# Step 11: Load the photos.tsv file containing 'property_id' and 'photo_urls'
photos_file_path = "property_photos.tsv"  # Adjust the path if needed
try:
    photos_df = pd.read_csv(photos_file_path, sep='\t')  # Assuming it's a TSV file
    print(f"Photos data loaded successfully from {photos_file_path}.")
except Exception as e:
    print(f"Error loading photos data: {e}")

# Check if the 'photo_urls' column exists in photos_df
if 'photo_urls' in photos_df.columns:
    # Step 12: Create photo_count feature
    photos_df['photo_count'] = photos_df['photo_urls'].apply(lambda x: len(str(x).split(',')) if pd.notnull(x) else 0)
    print("Photo count feature created successfully.")
else:
    print("'photo_urls' column not found in the Property_photos dataset.")

# Step 13: Merge the photos_df with merged_data on 'property_id'
merged_data = merged_data.merge(photos_df[['property_id', 'photo_count']], on='property_id', how='left')

# Step 14: Load the property_interactions dataset
interactions_file_path = "property_interactions.csv"  # Adjust the path if needed
try:
    interactions_df = pd.read_csv(interactions_file_path)
    print(f"Interactions data loaded successfully from {interactions_file_path}.")
except Exception as e:
    print(f"Error loading interactions data: {e}")

# Step 15: Calculate total_interactions for each property
total_interactions = interactions_df.groupby('property_id').size().reset_index(name='total_interactions')

# Step 16: Merge the total_interactions with the merged_data
merged_data = merged_data.merge(total_interactions, on='property_id', how='left')

# Step 17: Save the updated data with the new features
output_file = "Property_data/outputs/cleaned_property_data_with_features.csv"
merged_data.to_csv(output_file, index=False)

# Step 18: Print the updated dataset information
print(f"Data with new features ('photo_count', 'total_interactions') has been saved to '{output_file}'.")
print(f"Final data contains {merged_data.shape[0]} rows and {merged_data.shape[1]} columns.")
print("Missing values after feature engineering:")
print(merged_data.isnull().sum())


Zip file extracted successfully.
Found 64 CSV files at Property_data/Property_data\*.csv.
Files extracted: ['Akshayanagar.csv', 'Arekere.csv', 'Banashankari.csv', 'Basavanagudi.csv', 'Basaveshwar Nagar.csv', 'Begur.csv', 'Bellandur.csv', 'Bilekahalli.csv', 'Bommanahalli.csv', 'Brookefield.csv', 'BTM 2nd Stage.csv', 'BTM_Layout.csv', 'Chikkalasandra.csv', 'Doddanekundi.csv', 'Ejipura.csv', 'Electronic City.csv', 'Electronics City Phase 1.csv', 'Gottigere.csv', 'HBR Layout.csv', 'Hebbal.csv', 'Hongasandra.csv', 'Hoodi.csv', 'Horamavu.csv', 'Hosakerehalli.csv', 'HSR_Layout.csv', 'Hulimavu.csv', 'Indiranagar.csv', 'Jayanagar.csv', 'JP Nagar.csv', 'K.R Puram.csv', 'Kadugodi.csv', 'Kaggadasapura.csv', 'Kalyan Nagar.csv', 'Kammanahalli.csv', 'Kasavanahalli.csv', 'Kengeri Satellite Town.csv', 'Kengeri.csv', 'Konanakunte.csv', 'Koramangala.csv', 'Krishnarajapura.csv', 'Kumaraswamy Layout.csv', 'Lingarajapuram.csv', 'Mahadevapura.csv', 'Marathahalli.csv', 'Mathikere.csv', 'Munnekollal.csv', 'Nag